# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling DeepLearning.AI: A Revolution in Education

Dive into the world of artificial intelligence with DeepLearning.AI, a groundbreaking platform revolutionizing online learning. Led by AI expert Andrew Ng, this platform offers top-notch courses on deep learning, machine learning, and AI technology. With a user-friendly interface and engaging content, DeepLearning.AI makes complex concepts easily digestible for learners of all levels. Whether you're a novice or an industry professional, this platform provides invaluable knowledge to propel your career forward in the ever-evolving tech industry. Join the AI revolution today with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling DeepLearning.AI: A Revolution in Education

Dive into the world of artificial intelligence with DeepLearning.AI, a groundbreaking platform revolutionizing online learning. Led by AI expert Andrew Ng, this platform offers top-notch courses on deep learning, machine learning, and AI technology. With a user-friendly interface and engaging content, DeepLearning.AI makes complex concepts easily digestible for learners of all levels. Whether you're a novice or an industry professional, this platform provides invaluable knowledge to propel your career forward in the ever-evolving tech industry. Join the AI revolution today with DeepLearning.AI.

----------------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling DeepLearning.AI: A Revolution in Education

Dive into the world of artificial intelligence with DeepLearning.AI, a groundbreaking platform revolutionizing online learning. Led by AI expert Andrew Ng, this platform offers top-notch courses on deep learning, machine learning, and AI technology. With a user-friendly interface and engaging content, DeepLearning.AI makes complex concepts easily digestible for learners of all levels. Whether you're a novice or an industry professional, this platform provides invaluable knowledge to propel your career forward in the ever-evolving tech industry. Join the AI revolution today with DeepLearning.AI.

----------------------------------------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback from all reviewers:

1. **SEO Reviewer**: 
   - Use relevant keywords like artificial intelligence, Andrew Ng, DeepLearning.AI, and online learning for better search engine visibility.
   - Utilize meta tags and internal links to related content for improved website SEO performance.

2. **Legal Reviewer**: 
   - Ensure proper attribution for claims about DeepLearning.AI and Andrew Ng.
   - Avoid using unsubstantiated superlatives without evidence.
   - Include a disclaimer about career advancement not being guaranteed solely by participating in courses offered by DeepLearning.AI.

3. **Ethics Reviewer**:
   - Avoid using unsubstantiated superlatives.
   - Verify and attribute claims about DeepLearning.AI and Andrew Ng.
   - Include a disclaimer about career advancement in the tech industry.

**Final Suggestions**:
- Incorporate the relevant keywords suggeste

## Get the summary

In [15]:
print(res.summary)

Title: Navigating the World of Artificial Intelligence with DeepLearning.AI

Embark on your AI journey with DeepLearning.AI, an innovative online platform spearheaded by renowned expert Andrew Ng. Explore courses on artificial intelligence, machine learning, and deep learning in a user-friendly interface designed for all skill levels. Enhance your tech prowess and stay ahead in the industry with DeepLearning.AI's engaging content. Remember, career growth is a combination of learning and practical application - disclaimer. Join the AI evolution with DeepLearning.AI today for a brighter tomorrow.
